In [ ]:
import whisper
import subprocess
import datetime
from pathlib import Path


Hiperparámetros

In [ ]:
LANG = "es"
NORMALIZE_AUDIO = True
TEMPERATURE = 0.0
BEAM_SIZE = 8
INITIAL_PROMPT = (
    "Transcripción fiel en español de un archivo de audio. "
    "Usar ortografía y gramática correctas, con buena puntuación. "
    "Contexto: discurso, conferencia o grabación personal. "
    "Palabras clave: claridad, precisión, coherencia, fidelidad."
)

In [ ]:
# --- Funciones auxiliares ---
def toWav16k(inPath: Path, outPath: Path) -> Path:
    cmd = [
        "ffmpeg", "-y", "-i", str(inPath),
        "-vn",
        "-ac", "1",
        "-af", "aresample=resampler=soxr:precision=33",
        "-ar", "16000",
        "-c:a", "pcm_s16le",
        str(outPath)
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return outPath

def prepareAudioForWhisper(srcPath: Path, enabled: bool = True) -> Path:
    if not enabled:
        return srcPath
    target = srcPath.parent / "input.16k.wav"
    return toWav16k(srcPath, target)

def makeTimestamp() -> str:
    return datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# --- Uso ---
# Puedes cambiar "base" por "large-v3" o "turbo"
model = whisper.load_model("base")  

audioPath = Path("./audio.mp3")
normalizedPath = prepareAudioForWhisper(audioPath, enabled=NORMALIZE_AUDIO)

result = model.transcribe(
    str(normalizedPath),
    language=LANG,
    task="transcribe",
    temperature=TEMPERATURE,
    beam_size=BEAM_SIZE,
    patience=1.0,
    condition_on_previous_text=True,
    initial_prompt=INITIAL_PROMPT if INITIAL_PROMPT.strip() else None,
    fp16=False   # si CPU; en GPU puedes pasar True
)

text = result["text"].strip()

Guardar en archivo

In [ ]:
outFile = Path(f"transcript-{makeTimestamp()}.txt")
outFile.write_text(text, encoding="utf-8")
print(f"\n✅ Transcripción guardada en: {outFile}")

print("📄 Texto transcrito:\n")
print(text)